## Code to record a single PUMP/probe experiment 
Experimental setup is here: https://www.protocols.io/edit/pump-probe-experiment-cbvxsn7n 

and in this paper: 

Gallot, T., et al. "Characterizing the nonlinear interaction of S-and P-waves in a rock sample." Journal of Applied Physics 117.3 (2015): 034902.

In [2]:
#Import necessary files etc
from numpy import *
from pylab import *
import string

from datetime import date

#for connecting with the scope
import AgilentSCPI
import time
import numpy as np
import sys
import matplotlib.pyplot as plt
import time

### Parameter setup

In [2]:
delayvec=np.arange(0,90,1)
n_aves=4096
#2 ms between pulses, this is how long to wait to ensure that one signal is 
#recorded before sending another signal
t_delay=n_aves*2e-3 
addtodelay=54  #shift so that probe waveform is in the centre of the oscilloscope screen at the first delay
#Place to write data (note we will make sub-directories with parameters)
fname_root="/Users/amalcolm/Dropbox/Nonlinear/Experiments/MyData/WetBerea/Dry/ProbeOnTop/May13/"
ampval=8.0

8.192


### Classes for handling and storing the data

In [3]:
#For the data, store a time vector and data vector together
class timedata:

    def __init__(self, name):
        self.name = name
        self.time = np.array    # creates a new empty list
        self.data = np.array    # creates a new empty list


In [4]:
def readlabdat(fname,dataset):
# read in data from the scope and make a matrix of values from it
    
    
    f = open(fname,'r')
    tmp1=[]
    tmp2=[]
    
    for line in f:
        columns = line.split(',')
        tmp1.append(float(columns[0]))
        tmp2.append(float(columns[1]))

    dataset.time=np.asarray(tmp1)
    dataset.data=np.asarray(tmp2)
    f.close()
        

In [5]:
# read data from oscilloscope and save to file
def saveWave(filename,chs):
    for ch in chs:
        p1,x1 = osc.OscRead2Numpy(ch)
        if 'data' in locals():
            data = vstack((data,x1))
        else:
            data = x1
            
    
    t = arange(len(x1))*p1[4] + p1[5]
    data = vstack((t,data)).T
    savetxt(filename, data, fmt="%12.6G", delimiter=',')
    print(str(shape(data)), " data points saved to "+filename)

### Setup Function Generator

In [6]:
funcgen = AgilentSCPI.SCPI("134.153.37.159")  #IP address is available from the equipment
#Check its status
q=funcgen.query('*IDN?\n')
#Load the setup file
setupFile="STATE_BEREA_MAY13.STA"
cmd = ":MMEMory:LOAD:STATe 'INT:\\"+setupFile+"'\n"
funcgen.s.send(cmd)
#Set the amplitude of the PUMP (first) and then the probe
funcgen.s.send(":SOUR2:VOLT"+" "+str(ampval)+"\n")
funcgen.s.send(":SOUR1:VOLT"+" 5.0"+"\n")


Agilent Technologies,33522B,MY52802044,3.03-1.19-2.00-52-00



16

### Setup Oscilloscope

In [7]:
#IP address available from scope in the LAN setup 
osc = AgilentSCPI.SCPI("134.153.37.68")
#Check the connection
q2=osc.query('*IDN?\n')
print(q2)
#reset to get default values
osc.reset()
#Set trigger parameters
osc.s.send(":TRIGger:EDGE:SOURCe EXTernal\n")
osc.s.send(":TRIGger:EDGE:LEVel 5.0\n")
osc.s.send(":TRIGger:EDGE:SLOPe POSitive\n")
#Set time scale parameters
osc.s.send(":TIM:REF LEFT\n")
osc.s.send(":TIM:DEL 30us\n")
osc.s.send(":TIM:SCAL 0.5us\n")
#Set voltage scale parameters
osc.s.send(":CHANnel1:SCALe 0.01\n")
#Set acquisition and averaging parameters
osc.s.send(":ACQuire:TYPE AVERage\n")
osc.s.send(":ACQuire:COUNt 4096\n")

AGILENT TECHNOLOGIES,MSO-X 2014A,MY54100349,02.36.2013091301



20

### Run the experiment
First record the PUMP by itself, on a larger scale, then loop through all delays and collect all necessary signals.

In [8]:
#Set up the function generator so that only the PUMP is sent
funcgen.s.send(":SOUR2:VOLT"+" "+str(ampval)+"\n")
funcgen.s.send(":OUTP1 ON\n")
funcgen.s.send(":OUTP2 ON\n")
cmd=":TRIG:DEL "+str(0)+"us\n"
funcgen.s.send(cmd)
#Save the PUMP waveform from the oscilloscope
fname=fname_root+str("STrans_B4.csv")
osc.s.send(":TIM:REF LEFT\n")
osc.s.send(":TIM:SCAL 20us\n")
osc.s.send(":TIM:DEL 30us\n")
osc.s.send("CHANnel2:SCALe 25\n")
saveWave(fname,[2])
osc.s.send(":TIM:SCAL 1.0us\n")


scaldel=1  #parameter for changing the filename if there's a decimal in the delays
#Run through all of the delays and do the experiment
for delay in delayvec:
    #set delay
    cmd=":TRIG:DEL "+str(delay)+"us\n"
    funcgen.s.send(cmd)
    
    osc.s.send(":TIM:DEL "+ str(addtodelay+delay)+"us\n")

    #record probe only
    funcgen.s.send(":OUTP1 ON\n") #probe on
    funcgen.s.send(":OUTP2 OFF\n") #PUMP off

    time.sleep(t_delay)

    fname=fname_root+"pr"+str(int(scaldel*delay))+".csv"
    saveWave(fname,[1])

    #record PUMP only
    osc.s.send(":TIM:DEL "+ str(addtodelay+delay)+"us\n")
    funcgen.s.send(":OUTP1 OFF\n") #probe on
    funcgen.s.send(":OUTP2 ON\n") #PUMP off
    time.sleep(t_delay)

    fname=fname_root+"pu"+str(int(scaldel*delay))+".csv"
    saveWave(fname,[1])

    #record PUMP and probe
    osc.s.send(":TIM:DEL "+ str(addtodelay+delay)+"us\n")
    funcgen.s.send(":OUTP1 ON\n") #probe on
    funcgen.s.send(":OUTP2 ON\n") #PUMP off
    time.sleep(t_delay)

    fname=fname_root+"pp"+str(int(scaldel*delay))+".csv"
    saveWave(fname,[1])

#Turn off function generator so that we are not sending waves any longer
funcgen.s.send(":OUTP1 OFF\n")
funcgen.s.send(":OUTP2 OFF\n")

(7679, 2) data points saved to /Users/amalcolm/Dropbox/Nonlinear/Experiments/MyData/WetBerea/Dry/ProbeOnTop/May13/STrans_B4.csv
(2500, 2) data points saved to /Users/amalcolm/Dropbox/Nonlinear/Experiments/MyData/WetBerea/Dry/ProbeOnTop/May13/pr30.csv
(2500, 2) data points saved to /Users/amalcolm/Dropbox/Nonlinear/Experiments/MyData/WetBerea/Dry/ProbeOnTop/May13/pu30.csv
(2500, 2) data points saved to /Users/amalcolm/Dropbox/Nonlinear/Experiments/MyData/WetBerea/Dry/ProbeOnTop/May13/pp30.csv
(2500, 2) data points saved to /Users/amalcolm/Dropbox/Nonlinear/Experiments/MyData/WetBerea/Dry/ProbeOnTop/May13/pr31.csv
(2500, 2) data points saved to /Users/amalcolm/Dropbox/Nonlinear/Experiments/MyData/WetBerea/Dry/ProbeOnTop/May13/pu31.csv
(2500, 2) data points saved to /Users/amalcolm/Dropbox/Nonlinear/Experiments/MyData/WetBerea/Dry/ProbeOnTop/May13/pp31.csv
(2500, 2) data points saved to /Users/amalcolm/Dropbox/Nonlinear/Experiments/MyData/WetBerea/Dry/ProbeOnTop/May13/pr32.csv
(2500, 2) d

11